#### Audio Classification Data Preprocessing

In [2]:
### Let's read a sample audio using librosa
import librosa
audio_file_path='UrbanSound8K/Dog.wav'
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)

In [69]:
print(librosa_audio_data)

[ 0.00330576  0.00468387  0.00360455 ... -0.0037562  -0.00348641
 -0.00356705]


In [70]:
### Lets plot the librosa audio data
import matplotlib.pyplot as plt
# Original audio with 1 channel 
plt.figure(figsize=(12, 4))
plt.plot(librosa_audio_data)

#### Observation
Here Librosa converts the signal to mono, meaning the channel will alays be 1

In [3]:
### Lets read with scipy
from scipy.io import wavfile as wav
wave_sample_rate, wave_audio = wav.read(audio_file_path) 

In [4]:
wave_audio

array([[   0,    0],
       [   0,    0],
       [   0,    0],
       ...,
       [-399, -115],
       [-388, -111],
       [-386, -105]], dtype=int16)

In [5]:
import matplotlib.pyplot as plt

# Original audio with 2 channels 
plt.figure(figsize=(12, 4))
plt.plot(wave_audio)

### Extract Features
Here we will be using Mel-Frequency Cepstral Coefficients(MFCC) from the audio 
samples.
The MFCC summarises the frequency distribution across the window size, so it is possible to analyse both the frequency and time characteristics of the sound. These audio representations will allow us to identify features for classification.

In [6]:
mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)
print(mfccs.shape)

(40, 173)


In [7]:
mfccs

array([[-5.7041687e+02, -5.0580988e+02, -4.4661017e+02, ...,
        -4.8128598e+02, -4.7245932e+02, -4.6425366e+02],
       [ 3.2360130e+01,  9.3758621e+01,  1.4270895e+02, ...,
         1.0403226e+02,  1.1474982e+02,  1.1683809e+02],
       [ 2.6088989e+01,  3.4996727e+01,  4.0650406e+01, ...,
         1.1899885e+01,  1.7138729e+01,  1.3889361e+01],
       ...,
       [ 5.9291601e-01,  2.7261462e+00, -2.6107330e+00, ...,
        -2.7341046e+00, -3.7576494e+00, -5.8963108e+00],
       [-1.3069378e+00, -2.1206622e+00, -9.1924393e-01, ...,
        -1.2278577e+00, -3.3734822e+00, -4.1975555e+00],
       [-2.0638499e+00, -2.9027801e+00,  2.7278209e-01, ...,
        -1.3872120e+00, -3.3259065e+00, -8.4930527e-01]], dtype=float32)

In [8]:
#### Extracting MFCC's For every audio file
import pandas as pd
import os
import librosa

audio_dataset_path='UrbanSound8K/audio/'
metadata=pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
metadata.head()

slice_file_name    fsID  start        end  salience  fold  classID  \
0    100032-3-0-0.wav  100032    0.0   0.317551         1     5        3   
1  100263-2-0-117.wav  100263   58.5  62.500000         1     5        2   
2  100263-2-0-121.wav  100263   60.5  64.500000         1     5        2   
3  100263-2-0-126.wav  100263   63.0  67.000000         1     5        2   
4  100263-2-0-137.wav  100263   68.5  72.500000         1     5        2   

              class  
0          dog_bark  
1  children_playing  
2  children_playing  
3  children_playing  
4  children_playing

In [9]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features
    

In [10]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

3554it [05:02, 12.60it/s]d:\Useful\Projects\Audio_Classification\.venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
8325it [11:21, 18.39it/s]d:\Useful\Projects\Audio_Classification\.venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
d:\Useful\Projects\Audio_Classification\.venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
8732it [11:51, 12.27it/s]


In [11]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

feature             class
0  [-217.35526, 70.22339, -130.38527, -53.282898,...          dog_bark
1  [-424.09818, 109.34077, -52.919525, 60.86475, ...  children_playing
2  [-458.79114, 121.38419, -46.520657, 52.00812, ...  children_playing
3  [-413.89984, 101.66371, -35.42945, 53.036354, ...  children_playing
4  [-446.60352, 113.68541, -52.402218, 60.302044,...  children_playing

In [12]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [13]:
X.shape

(8732, 40)

In [14]:
y

array(['dog_bark', 'children_playing', 'children_playing', ...,
       'car_horn', 'car_horn', 'car_horn'], dtype='<U16')

In [15]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [16]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [17]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [18]:
X_train

array([[-1.3110471e+02,  1.1250591e+02, -2.2574694e+01, ...,
         3.2466519e+00, -1.3690238e+00,  2.7557547e+00],
       [-1.3670342e+01,  9.1085083e+01, -7.7927332e+00, ...,
        -3.2530508e+00, -5.2774525e+00, -1.5569715e+00],
       [-4.9871544e+01,  2.6535299e-01, -2.0500937e+01, ...,
         2.8545945e+00, -1.6092044e+00,  3.5248058e+00],
       ...,
       [-4.2701236e+02,  9.2623047e+01,  3.1293974e+00, ...,
         7.4264121e-01,  7.3349077e-01,  7.1100914e-01],
       [-1.4575461e+02,  1.3626578e+02, -3.3515522e+01, ...,
         1.4681195e+00, -2.0091701e+00, -8.8218188e-01],
       [-4.2103134e+02,  2.1065454e+02,  3.4906607e+00, ...,
        -5.3888674e+00, -3.3713605e+00, -1.5665114e+00]], dtype=float32)

In [19]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [20]:
X_train.shape

(6985, 40)

In [21]:
X_test.shape

(1747, 40)

In [22]:
y_train.shape

(6985, 10)

In [23]:
y_test.shape

(1747, 10)

### Model Creation

In [24]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [26]:
### No of classes
num_labels=y.shape[1]

In [27]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

d:\Useful\Projects\Audio_Classification\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │         4,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,410 (177.38 KB)

 Trainable params: 45,410 (177.38 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [40]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 150
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.keras', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/150
193/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6726 - loss: 0.9675
Epoch 1: val_loss improved from inf to 0.78270, saving model to saved_models/audio_classification.keras
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6727 - loss: 0.9682 - val_accuracy: 0.7613 - val_loss: 0.7827
Epoch 2/150
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6759 - loss: 0.9782
Epoch 2: val_loss improved from 0.78270 to 0.78081, saving model to saved_models/audio_classification.keras
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6759 - loss: 0.9782 - val_accuracy: 0.7481 - val_loss: 0.7808
Epoch 3/150
196/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6685 - loss: 0.9904
Epoch 3: val_loss did not improve from 0.78081
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6697 - loss: 0.9891 - val_accuracy: 0.7384 - val_loss: 0.8105
Epoch 4/150
206/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6857 - loss: 0.9425
Epoch 4: val_loss improved from 0.78081 to

In [41]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8042358160018921


In [43]:
X_test[1]

array([-466.17957   ,    1.0950246 ,  -34.01389   ,   35.339348  ,
        -14.88148   ,  -19.12843   ,   -0.5816844 ,  -16.130579  ,
        -21.339073  ,    7.673634  ,  -29.164494  ,  -18.950253  ,
         -2.9579997 ,   -8.162329  ,  -15.153101  ,   -6.6048055 ,
         -7.568598  ,    9.340646  ,   14.4331    ,   21.934181  ,
         20.861397  ,    1.3340123 ,  -19.228804  ,   -4.6302314 ,
         -1.0564744 ,    3.215267  ,   -6.9842815 ,  -16.414577  ,
        -10.0286455 ,   13.009954  ,    0.53346086,  -23.843391  ,
        -15.267322  ,    9.245734  ,   10.367628  ,   -0.5832011 ,
         -1.2624054 ,   17.700016  ,   13.847463  ,   -5.1862826 ],
      dtype=float32)

In [44]:
model.predict(X_test)

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[1.24585549e-05, 3.38338927e-04, 6.50583825e-04, ...,
        1.10117815e-09, 1.13523405e-04, 3.51238344e-03],
       [2.48402771e-10, 1.26929590e-02, 1.23559303e-05, ...,
        2.53269286e-06, 3.01529672e-05, 2.51148333e-04],
       [1.59239146e-08, 2.46276497e-03, 1.67891594e-05, ...,
        1.06396918e-04, 3.19137303e-06, 3.86052910e-04],
       ...,
       [1.48865396e-07, 9.76821959e-01, 4.96354096e-06, ...,
        3.71063935e-09, 9.89937689e-06, 1.54012740e-02],
       [6.00963086e-02, 5.49634080e-03, 4.24420267e-01, ...,
        3.66174174e-03, 1.55074000e-02, 3.68830174e-01],
       [1.01946250e-01, 1.63708273e-02, 1.38869777e-01, ...,
        5.64523134e-03, 3.61385420e-02, 3.43456805e-01]], dtype=float32)

### Testing Some Test Audio Data

Steps
- Preprocess the new audio data
- Predict the classes


In [49]:
filename = "UrbanSound8K/music.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features = mfccs_scaled_features.reshape(1, -1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)

predicted_probabilities = model.predict(mfccs_scaled_features)
print(predicted_probabilities)

# Get the index of the class with the highest probability
predicted_label_index = np.argmax(predicted_probabilities, axis=1)
print(predicted_label_index)

# Inverse transform to get the actual label
prediction = labelencoder.inverse_transform(predicted_label_index) 
print(prediction)


[-3.2884320e+02  1.5263676e+02 -1.7522491e+01  4.0471443e+01
 -2.4742644e+00 -4.8075643e+00  9.5028200e+00  6.9135580e+00
  1.0047156e+01  2.5325179e+00  1.6533588e+01  2.0937944e+01
  3.6484206e+00 -5.8015537e+00  3.1274450e-01  9.0401802e+00
  9.3883953e+00  4.0399389e+00 -3.6789987e+00 -2.4209588e+00
 -1.1935220e+00  8.4478722e+00  8.7916412e+00  3.9289465e+00
  1.2568516e+00 -4.6916356e+00 -9.6353376e-01  3.8441839e+00
 -1.7103158e-01 -3.7653744e+00  2.0401618e+00  9.7673187e+00
  7.0352354e+00 -9.7400159e-01  4.8541188e-01 -9.0075523e-01
 -9.4677534e+00 -5.3134475e+00  6.5572023e+00  2.4861794e+00]
[[-3.2884320e+02  1.5263676e+02 -1.7522491e+01  4.0471443e+01
  -2.4742644e+00 -4.8075643e+00  9.5028200e+00  6.9135580e+00
   1.0047156e+01  2.5325179e+00  1.6533588e+01  2.0937944e+01
   3.6484206e+00 -5.8015537e+00  3.1274450e-01  9.0401802e+00
   9.3883953e+00  4.0399389e+00 -3.6789987e+00 -2.4209588e+00
  -1.1935220e+00  8.4478722e+00  8.7916412e+00  3.9289465e+00
   1.2568516e+00 